In [ ]:
pip install transformers

In [ ]:
import sys
sys.path.append('..')

In [2]:
from Aspect_ZeroShot_Baseline import Aspect_QAModel
from transformers import  AutoTokenizer, OPTForCausalLM, AutoModel, GPT2Tokenizer, GPT2LMHeadModel
from helper import *
import torch
import numpy as np
from sklearn.utils import shuffle
import json

In [5]:
config_path = "../config.json"
all_data, folds = load_config(config_path)

train_inds = folds[0][0]
test_inds = folds[0][2]

test_data = [all_data[i] for i in test_inds]

In [6]:
def custom_gmean(lst):
  gmean = np.prod(lst)**(1/len(lst))
  return gmean

In [7]:
def ZeroShot_QA(model, data, agg_fcn):
  correct = 0
  type_correct = {
      "Specific": 0,
      "Subjective": 0,
      "Indirect": 0,
      "Compound": 0,
      "Negated": 0,
      "Analogical": 0,
      "Temporal": 0}
  
  for d in data:
    options_list = [val for val in d['options'].values()]
    aspects = d['correctness_explanation'].keys()
    correct_answer = d['options'][d['answer']]
    all_scores = []

    for a in aspects:
      scores = model.get_answer(a, options_list)
      all_scores.append(scores)

    agg_scores = aggregate(all_scores, agg_fcn)
    agg_scores, options_list = shuffle(agg_scores, options_list, random_state=0)
    args = np.argsort(agg_scores)
    answer = options_list[args[-1]]

    if answer == correct_answer:
      correct += 1
      for key in d['query_type']:
        if d['query_type'][key] == 1:
          type_correct[key] += 1

  print("Total correct: ", correct)
  print(type_correct)

In [ ]:
model_name = config['zeroshot_model']
agg_fcn = min

model, tokenizer = get_model(model_name)

qa_model = Aspect_QAModel(model, tokenizer, model_name)
ZeroShot_QA(qa_model, test_data, agg_fcn)